In [7]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import warnings

import pandas_ta as ta
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

In [9]:
train = pd.read_pickle("train_기술적_분석.pkl")
test = pd.read_pickle("prediction_기술적_분석.pkl")

In [12]:
train.head(1)

,ticker,firm,volume,open,high,low,close,adjustTrue,SMA_10,SMA_20,...,BBP_5_2.0,ADX_14,DMP_14,DMN_14,EFI_13,KAMA_10_2_30,MFI_14,VTXP_14,VTXM_14,target
date,,,,,,,,,,,,,,,,,,,,,
2022-03-23,A000020,동화약품,396150.0,13800.0,14100.0,13600.0,13650.0,1,13335.0,13020.0,...,0.360653,20.440277,30.358104,13.516124,2.075979e+08,13031.963084,74.680545,1.109091,0.842424,12800.0


In [20]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
from tqdm import tqdm
from joblib import Parallel, delayed

count = 0

# XGBoost 하이퍼파라미터의 탐색 공간 정의
# param_grid = {
#     'n_estimators': [100, 200, 500, 1000],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
#     'min_child_weight': [1, 3, 5],
#     'subsample': [0.5, 0.7, 1],
#     'colsample_bytree': [0.5, 0.7, 1]
# }

param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.1],
}

results_df = pd.DataFrame(columns=['ticker', 'final_return'])

# 특정 주식에 대해 Grid Search를 수행하는 함수
def grid_search_for_one_stock(ticker, task:pd.DataFrame):
    # 특정 주식 데이터만 선택
    data = train[train['ticker'] == ticker]
    task = test[test['ticker'] == ticker]

    # 특성 및 타겟 정의
    X = data.drop(['ticker', 'firm', 'target'], axis=1)
    y = data['target']

    X_pred = task.drop(['ticker', 'firm', 'target'], axis=1)

    # XGBoost 모델 생성
    model = XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist')
    model.fit(X, y)
    # Grid Search 실행
    #tscv = TimeSeriesSplit(n_splits=5)
    #grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_mean_squared_error')
    #grid_search.fit(X, y)

    # 최적 하이퍼파라미터를 사용하여 학습
    #best_model = grid_search.best_estimator_
    #best_model.fit(X, y)

    # 향후 14개의 거래일에 대해서 예측
    predictions = model.predict(X_pred)

    # 최종 수익률 계산
    final_return = (predictions[-1] - predictions[0]) / predictions[0]

    print(count, "/", len(train), ticker, final_return)

    # 결과 저장
    return ticker, final_return

# 모든 주식에 대해 병렬로 Grid Search 실행
results = Parallel(n_jobs=-1)(delayed(grid_search_for_one_stock)(ticker, test) for ticker in train['ticker'].unique())

# 결과 출력 및 저장
for ticker, final_return in results:
    print(f"Final return for {ticker}: {final_return}")
    results_df = results_df.append({'ticker': ticker, 'final_return': final_return}, ignore_index=True)


KeyboardInterrupt: 